# 爬取办公资源上的文档
悲剧的我被反爬了
![](http://881023.top/image/被发现了.png)

In [ ]:
import os
from requests_html import HTMLSession
import urllib
import urllib.request
import requests
# https
import ssl
ssl._create_default_https_context=ssl._create_unverified_context
# 线程池
from concurrent.futures import ThreadPoolExecutor,wait,as_completed
import json

In [ ]:
session = HTMLSession()
# 创建一个ua，绕过初级反爬
ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:62.0) Gecko/20100101 Firefox/62.0'
headers = {'User-Agent': ua}
# 创建一个最大数量为3的线程池
executor = ThreadPoolExecutor(max_workers=3)

In [113]:
# 用户信息，json串格式, 编码问题，实在难搞
userInfo = '{"id":"93000","email":"","qq_loginkey":"","qq_account":"","weixin_loginkey":"ow7770tQu8RGQEMY483H32TsKi2M","weixin_account":"L55QD-ZmXC_lOsBz7NdR0eVY1zrp8mAYFlyNwCqA","img":"http://thirdwx.qlogo.cn/mmopen/vi_32/Q0j4TwGTfTJXM9QWK5dWnj2gfutX4TGicb5DMlZticBiajaydMbzXFVhtdiclKb10c0GMzYLQQKd4Qs8d99lTf45NQ/132","name":"樱花残月","nickname":"","mobile_phone":"17625930265","member_type":"10","vip_expire_date":"2021-09-02 15:47:58","vip_first_expire_date":"2021-09-02 15:47:58","vip_first_member_type":"10","last_callback_trade_no":"180902034736638632","last_pay_date":"2018-09-02 15:47:58","last_pay_member_type":"10","guid":"6ead0f350bd4fc63f80bf4aaa71c738046c0b8cc","password":"5fded8de87c8f7d2e0b492fc8c37cecbdc72e3b2"}'
userInfoStr = '%7B%22id%22%3A%2293000%22%2C%22email%22%3A%22%22%2C%22qq_loginkey%22%3A%22%22%2C%22qq_account%22%3A%22%22%2C%22weixin_loginkey%22%3A%22ow7770tQu8RGQEMY483H32TsKi2M%22%2C%22weixin_account%22%3A%22L55QD-ZmXC_lOsBz7NdR0eVY1zrp8mAYFlyNwCqA%22%2C%22img%22%3A%22http%3A%2F%2Fthirdwx.qlogo.cn%2Fmmopen%2Fvi_32%2FQ0j4TwGTfTJXM9QWK5dWnj2gfutX4TGicb5DMlZticBiajaydMbzXFVhtdiclKb10c0GMzYLQQKd4Qs8d99lTf45NQ%2F132%22%2C%22name%22%3A%22%E6%A8%B1%E8%8A%B1%E6%AE%8B%E6%9C%88%22%2C%22nickname%22%3A%22%22%2C%22mobile_phone%22%3A%2217625930265%22%2C%22member_type%22%3A%2210%22%2C%22vip_expire_date%22%3A%222021-09-02%2015%3A47%3A58%22%2C%22vip_first_expire_date%22%3A%222021-09-02%2015%3A47%3A58%22%2C%22vip_first_member_type%22%3A%2210%22%2C%22last_callback_trade_no%22%3A%22180902034736638632%22%2C%22last_pay_date%22%3A%222018-09-02%2015%3A47%3A58%22%2C%22last_pay_member_type%22%3A%2210%22%2C%22guid%22%3A%226ead0f350bd4fc63f80bf4aaa71c738046c0b8cc%22%2C%22password%22%3A%225fded8de87c8f7d2e0b492fc8c37cecbdc72e3b2%22%7D'
# 解析用户信息json
userObject = json.loads(userInfo)
# cookie
cookies = {'userInfo': userInfoStr}

要用的方法

In [141]:
# 获取总页数
def get_total_page_count():
    response = session.get('http://www.bangongziyuan.com/ppt/pn/1.html', headers=headers, cookies=cookies)
    # 获取当前一共多少页
    pages = response.html.find('input#pptAll', first=True)
    pageCount = int(pages.attrs['value'])
    return pageCount
# 获取后半部分文件地址
def get_file_path(proId):
    downloadResp = session.post(downloadUrl, {'proId':proId, 'userId':userObject['id']})
    fileWholeObject = json.loads(downloadResp.html.text)
    # 正常状态
    if fileWholeObject['status'] == 1:
        # 下载地址后半部分文件地址
        fileUrl = fileWholeObject['data']['path']
        return fileUrl
    # 估计是被限制了
    else:
        print('\033[1;31m 应该是被限制了 \033[0m')
        return ''
# 获取文件完整地址
def get_file_url(filePartUrl):
    if filePartUrl == '':
        return ''
    # 用来请求文档的相对地址的，下载地址,找得我好辛苦啊-。-
    downloadUrl = 'http://www.bangongziyuan.com/inter/addDownload'
    # 拼接下载地址的前半部分
    responseFirst = session.get('http://www.bangongziyuan.com/ppt/pn/1.html', headers=headers, cookies=cookies)
    fileFrontPart = responseFirst.html.find('input#fileUrl', first=True).attrs['value'].replace('http://image1.bangongziyuan.com','')
    print('完整的地址%s' % (fileFrontPart + filePartUrl.replace))
    return fileFrontPart + filePartUrl

# 从网页中摘取所有的proid,返回的是二维list
def get_file_id(url):
    result = []
    response = session.get(url, headers=headers, cookies=cookies)
    content = response.html.find('div.model-content', first=True)
    lists = content.find('div.model-item')
    for one in lists:
        name = one.find('a.title', first=True).text
        id = one.find('div.down', first=True).attrs['data-proid']
        result.append([name, id])
    return result

# 下载文件
def downloadfile(url, filename):
    print('开始下载文件')
    try:
        # r = requests.get(url)
        path = "docs/"+filename
        r = requests.get(url, headers=headers)
        with open(path,"wb") as f:
            f.write(r.content)
        f.close()
    except Exception:
        print('\033[1;31m 下载失败 \033[0m')
    else:
        print("\033[1;32m 下载", filename, "完成\033[0m")

爬取流程

In [ ]:
pageCount = get_total_page_count()
for index in range(pageCount):
    pageUrl = 'http://www.bangongziyuan.com/ppt/pn/%d.html' % (index+1)
    print(pageUrl)
    docs = get_file_id(pageUrl)
    for docInfo in docs:
        start_one_doc_get(docInfo)
    

In [143]:
# 开始获取一个文件
def start_one_doc_get(docInfo):
    title = docInfo[0]
    id = docInfo[1]
    filePart = get_file_path(id)
    fileWholeUrl = get_file_url(filePart)
    print(fileWholeUrl)

In [ ]:
get_file_id('http://www.bangongziyuan.com/ppt/pn/1.html')